<a href="https://colab.research.google.com/github/microsoft/qlib/blob/main/examples/workflow_by_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import sys
import pandas as pd

import qlib
from qlib.config import REG_CN
from qlib.contrib.data.handler import Alpha158
from qlib.contrib.evaluate import (
    backtest as normal_backtest,
    risk_analysis,
)
from qlib.utils import exists_qlib_data, init_instance_by_config
from qlib.workflow import R
from qlib.workflow.record_temp import SignalRecord, PortAnaRecord
from qlib.utils import flatten_dict

In [8]:
provider_uri = "../../data/qlib_data_v2"
scripts_dir = "../../scripts"
if not exists_qlib_data(provider_uri):
    print(f"Qlib data is not found in {provider_uri}")
    sys.path.append(str(scripts_dir))
    from get_data import GetData
    GetData().qlib_data(target_dir=provider_uri, region=REG_CN)
qlib.init(provider_uri=provider_uri, region=REG_CN)

[4704:MainThread](2021-11-05 19:00:13,593) INFO - qlib.Initialization - [config.py:337] - default_conf: client.
[4704:MainThread](2021-11-05 19:00:13,595) WARNING - qlib.Initialization - [config.py:352] - redis connection failed(host=127.0.0.1 port=6379), cache will not be used!
[4704:MainThread](2021-11-05 19:00:13,597) INFO - qlib.Initialization - [__init__.py:58] - qlib successfully initialized based on client settings.
[4704:MainThread](2021-11-05 19:00:13,598) INFO - qlib.Initialization - [__init__.py:60] - data_path={'__DEFAULT_FREQ': PosixPath('/data3/xujianjin/qlib/data/qlib_data_v2')}


In [9]:
os.path.abspath("./mlruns")

'/data3/xujianjin/qlib/examples/project2/mlruns'

# analyze graphs

In [10]:
from qlib.contrib.report import analysis_model, analysis_position
from qlib.data import D
recorder = R.get_recorder(recorder_id="94961084bb8649bdb9707a809276a7af", experiment_name="backtest_analysis")
print(recorder)
pred_df = recorder.load_object("pred.pkl")
pred_df_dates = pred_df.index.get_level_values(level='datetime')
report_normal_df = recorder.load_object("portfolio_analysis/report_normal_1day.pkl")
positions = recorder.load_object("portfolio_analysis/positions_normal_1day.pkl")
analysis_df = recorder.load_object("portfolio_analysis/port_analysis_1day.pkl")

{'class': 'Recorder', 'id': '94961084bb8649bdb9707a809276a7af', 'name': 'mlflow_recorder', 'experiment_id': '2', 'start_time': '2021-11-05 18:13:27', 'end_time': '2021-11-05 18:13:28', 'status': 'FAILED'}


LoadObjectError: No such file or directory: '/data3/xujianjin/qlib/examples/project2/mlruns/2/94961084bb8649bdb9707a809276a7af/artifacts/portfolio_analysis/report_normal_1day.pkl'

## analysis position

### report

In [ ]:
analysis_position.report_graph(report_normal_df)

### risk analysis

In [ ]:
analysis_position.risk_analysis_graph(analysis_df, report_normal_df)

## analysis model

In [ ]:
label_df = dataset.prepare("test", col_set="label")
label_df.columns = ['label']

### score IC

In [ ]:
pred_label = pd.concat([label_df, pred_df], axis=1, sort=True).reindex(label_df.index)
analysis_position.score_ic_graph(pred_label)

In [ ]:
analysis_model.model_performance_graph

### model performance

In [ ]:
analysis_model.model_performance_graph(pred_label)
# 选股的相关性 - 更高级的策略